In [19]:
import ROOT
from ROOT import RooFit, RooRealVar, RooDataSet, RooArgList, RooAddPdf, RooGaussian, RooFormulaVar

# Create variables
x = RooRealVar("x", "x", -10, 10)
Nsig_D_plus = RooRealVar("Nsig_D_plus", "Nsig D+", 1000, 0, 10000)
Nsig_D_minus = RooRealVar("Nsig_D_minus", "Nsig D-", 1000, 0, 10000)

# Define Acp as a formula based on Nsig_D_plus and Nsig_D_minus
Acp = RooFormulaVar("Acp", "(@0 - @1) / (@0 + @1)", RooArgList(Nsig_D_plus, Nsig_D_minus))

# Example Gaussian for D+ and D- signal
mean_D_plus = RooRealVar("mean_D_plus", "Mean D+", 0)
sigma_D_plus = RooRealVar("sigma_D_plus", "Sigma D+", 1)
gauss_D_plus = RooGaussian("gauss_D_plus", "Gaussian PDF for D+", x, mean_D_plus, sigma_D_plus)

mean_D_minus = RooRealVar("mean_D_minus", "Mean D-", 0)
sigma_D_minus = RooRealVar("sigma_D_minus", "Sigma D-", 1)
gauss_D_minus = RooGaussian("gauss_D_minus", "Gaussian PDF for D-", x, mean_D_minus, sigma_D_minus)

# Combine PDFs with extended likelihood
model_D_plus = RooAddPdf("model_D_plus", "D+ model", RooArgList(gauss_D_plus), RooArgList(Nsig_D_plus))
model_D_minus = RooAddPdf("model_D_minus", "D- model", RooArgList(gauss_D_minus), RooArgList(Nsig_D_minus))

# Simultaneous fit
sim_model = RooAddPdf("sim_model", "Simultaneous model", RooArgList(model_D_plus, model_D_minus), RooArgList(Nsig_D_plus, Nsig_D_minus))

# Create datasets (for example purposes, use random data)
data_D_plus = RooDataSet("data_D_plus", "D+ data", RooArgList(x))
data_D_minus = RooDataSet("data_D_minus", "D- data", RooArgList(x))

# Generate example data
for i in range(1000):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_plus.add(RooArgList(x))

for i in range(1000):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_minus.add(RooArgList(x))

# Combine datasets
data_combined = RooDataSet("data_combined", "Combined data", RooArgList(x))
data_combined.append(data_D_plus)
data_combined.append(data_D_minus)

# Fit the model to the data
fit_result = sim_model.fitTo(data_combined, RooFit.Save())

# Print fit results
fit_result.Print()

# Output the Acp value
Acp_value = Acp.getVal()

# Calculate error on Acp using the covariance matrix
cov_matrix = fit_result.covarianceMatrix()
error_D_plus = Nsig_D_plus.getError()
error_D_minus = Nsig_D_minus.getError()

# Calculate the derivative of Acp w.r.t Nsig_D_plus and Nsig_D_minus
dAcp_dNsig_D_plus = 2 * Nsig_D_minus.getVal() / (Nsig_D_plus.getVal() + Nsig_D_minus.getVal())**2
dAcp_dNsig_D_minus = -2 * Nsig_D_plus.getVal() / (Nsig_D_plus.getVal() + Nsig_D_minus.getVal())**2

# Propagate errors
Acp_error = (dAcp_dNsig_D_plus**2 * error_D_plus**2 +
             dAcp_dNsig_D_minus**2 * error_D_minus**2 +
             2 * cov_matrix[0][1] * dAcp_dNsig_D_plus * dAcp_dNsig_D_minus)

# Print results
print(f"Acp = {Acp_value:.3f} ± {Acp_error**0.5:.3f}")


Acp = 0.000 ± 1.648
[#0] WARNING:InputArguments -- The parameter 'sigma_D_plus' with range [-1e+30, 1e+30] of the RooGaussian 'gauss_D_plus' exceeds the safe range of (0, inf). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'sigma_D_minus' with range [-1e+30, 1e+30] of the RooGaussian 'gauss_D_minus' exceeds the safe range of (0, inf). Advise to limit its range.
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 Nsig_D_minus   1.00000e+03  5.00000e+02    0.00000e+00  1.00000e+04
     2 Nsig_D_plus   1.00000e+03  5.00000e+02    0.00000e+00  1.00000e+04
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000     

In [2]:
import ROOT
from ROOT import RooFit, RooRealVar, RooDataSet, RooArgList, RooAddPdf, RooGaussian, RooFormulaVar, RooSimultaneous, RooCategory

# Create variables
x = RooRealVar("x", "x", -10, 10)
Nsig_D_plus = RooRealVar("Nsig_D_plus", "Nsig D+", 1000, 0, 10000)
Acp = RooRealVar("Acp", "Acp", 0, -1, 1)  # A_Cp as a fit parameter

# Use Acp to define the expected signal yields for D-
Nsig_D_minus = RooFormulaVar("Nsig_D_minus", 
    "Nsig_D_plus * (1 - Acp) / (1 + Acp)", 
    RooArgList(Nsig_D_plus, Acp))

# Example Gaussian for D+ and D- signal
mean_D_plus = RooRealVar("mean_D_plus", "Mean D+", 0, -0.5, 0.5)
sigma_D_plus = RooRealVar("sigma_D_plus", "Sigma D+", 1, 0.1, 2)
gauss_D_plus = RooGaussian("gauss_D_plus", "Gaussian PDF for D+", x, mean_D_plus, sigma_D_plus)

mean_D_minus = RooRealVar("mean_D_minus", "Mean D-", 0, -0.5, 0.5)
sigma_D_minus = RooRealVar("sigma_D_minus", "Sigma D-", 1, 0.1, 2)
gauss_D_minus = RooGaussian("gauss_D_minus", "Gaussian PDF for D-", x, mean_D_minus, sigma_D_minus)

# Define extended PDFs for D+ and D-
model_D_plus = RooAddPdf("model_D_plus", "D+ model", RooArgList(gauss_D_plus), RooArgList(Nsig_D_plus))
model_D_minus = RooAddPdf("model_D_minus", "D- model", RooArgList(gauss_D_minus), RooArgList(Nsig_D_minus))

# Create a category to distinguish between D+ and D-
cat = RooCategory("sample", "sample")
cat.defineType("D_plus")
cat.defineType("D_minus")

# Create a simultaneous PDF using the category
sim_model = RooSimultaneous("sim_model", "Simultaneous model", cat)
sim_model.addPdf(model_D_plus, "D_plus")
sim_model.addPdf(model_D_minus, "D_minus")

# Create datasets for D+ and D- (without the Import statement)
data_D_plus = RooDataSet("data_D_plus", "D+ data", RooArgList(x))
data_D_minus = RooDataSet("data_D_minus", "D- data", RooArgList(x))

# Generate example data for D+ and D-
for i in range(500):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_plus.add(RooArgList(x))

for i in range(500):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_minus.add(RooArgList(x))

# Combine datasets into one, using RooFit.Import properly
data_combined = RooDataSet("data_combined", "Combined data", RooArgList(x), RooFit.Index(cat),
                           RooFit.Import("D_plus", data_D_plus),
                           RooFit.Import("D_minus", data_D_minus))

# Fit the model to the combined data
fit_result = sim_model.fitTo(data_combined, RooFit.Save())

# Print fit results
fit_result.Print()

# Output the Acp value and its error
Acp_value = Acp.getVal()
Acp_error = Acp.getError()

print(f"Acp = {Acp_value:.3f} ± {Acp_error:.3f}")


Acp = -0.000 ± 0.032
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
RooAbsTestStatistic::initSimMode: creating slave calculator #0 for state D_minus (500 dataset entries)
[#0] ERROR:Integration --  RooNumIntFactory::Init : libRooFitMore cannot be loaded. GSL integrators will not beavailable ! 
RooAbsTestStatistic::initSimMode: creating slave calculator #1 for state D_plus (500 dataset entries)
[#1] INFO:Fitting -- RooAbsTestStatistic::initSimMode: created 2 slave calculators.
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (gauss_D_minus)
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (gauss_D_plus)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME

cling::DynamicLibraryManager::loadLibrary(): libgsl.so.25: cannot open shared object file: No such file or directory


In [9]:
import ROOT
from ROOT import RooFit, RooRealVar, RooDataSet, RooArgList, RooAddPdf, RooGaussian, RooFormulaVar, RooSimultaneous, RooCategory

# Create variables
x = RooRealVar("x", "x", -10, 10)
N_total = RooRealVar("N_total", "N_total (N_D+ + N_D-)", 2000, 0, 20000)  # N_total = N_D+ + N_D-
Acp = RooRealVar("Acp", "Acp", 0, -1, 1)  # A_Cp as a fit parameter

# Use Acp and N_total to define the expected signal yields for D+ and D-
Nsig_D_plus = RooFormulaVar("Nsig_D_plus", 
    "0.5 * N_total * (1 + Acp)", 
    RooArgList(N_total, Acp))

Nsig_D_minus = RooFormulaVar("Nsig_D_minus", 
    "0.5 * N_total * (1 - Acp)", 
    RooArgList(N_total, Acp))

# Example Gaussian for D+ and D- signal
mean_D_plus = RooRealVar("mean_D_plus", "Mean D+", 0, -0.5, 0.5)
sigma_D_plus = RooRealVar("sigma_D_plus", "Sigma D+", 1, 0.1, 2)
gauss_D_plus = RooGaussian("gauss_D_plus", "Gaussian PDF for D+", x, mean_D_plus, sigma_D_plus)

mean_D_minus = RooRealVar("mean_D_minus", "Mean D-", 0, -0.5, 0.5)
sigma_D_minus = RooRealVar("sigma_D_minus", "Sigma D-", 1, 0.1, 2)
gauss_D_minus = RooGaussian("gauss_D_minus", "Gaussian PDF for D-", x, mean_D_minus, sigma_D_minus)

# Define extended PDFs for D+ and D-
model_D_plus = RooAddPdf("model_D_plus", "D+ model", RooArgList(gauss_D_plus), RooArgList(Nsig_D_plus))
model_D_minus = RooAddPdf("model_D_minus", "D- model", RooArgList(gauss_D_minus), RooArgList(Nsig_D_minus))

# Create a category to distinguish between D+ and D-
cat = RooCategory("sample", "sample")
cat.defineType("D_plus")
cat.defineType("D_minus")

# Create a simultaneous PDF using the category
sim_model = RooSimultaneous("sim_model", "Simultaneous model", cat)
sim_model.addPdf(model_D_plus, "D_plus")
sim_model.addPdf(model_D_minus, "D_minus")

# Create datasets for D+ and D- (without the Import statement)
data_D_plus = RooDataSet("data_D_plus", "D+ data", RooArgList(x))
data_D_minus = RooDataSet("data_D_minus", "D- data", RooArgList(x))

# Generate example data for D+ and D-
for i in range(520):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_plus.add(RooArgList(x))

for i in range(480):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_minus.add(RooArgList(x))

# Combine datasets into one, using RooFit.Import properly
data_combined = RooDataSet("data_combined", "Combined data", RooArgList(x), RooFit.Index(cat),
                           RooFit.Import("D_plus", data_D_plus),
                           RooFit.Import("D_minus", data_D_minus))

# Fit the model to the combined data
fit_result = sim_model.fitTo(data_combined, RooFit.Save())

# Print fit results
fit_result.Print()

# Output the Acp value and its error
Acp_value = Acp.getVal()
Acp_error = Acp.getError()

print(f"Acp = {Acp_value:.3f} ± {Acp_error:.3f}")

# Output N_total
N_total_value = N_total.getVal()
N_total_error = N_total.getError()

print(f"N_total = {N_total_value:.0f} ± {N_total_error:.3f}")


Acp = 0.040 ± 0.032
N_total = 1000 ± 31.623
[#1] INFO:Minimization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
RooAbsTestStatistic::initSimMode: creating slave calculator #0 for state D_minus (480 dataset entries)
RooAbsTestStatistic::initSimMode: creating slave calculator #1 for state D_plus (520 dataset entries)
[#1] INFO:Fitting -- RooAbsTestStatistic::initSimMode: created 2 slave calculators.
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (gauss_D_minus)
[#1] INFO:Minimization --  The following expressions will be evaluated in cache-and-track mode: (gauss_D_plus)
 **********
 **   37 **SET PRINT           1
 **********
 **********
 **   38 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 Acp          0.00000e+00  2.00000e-01   -1.00000e+00

In [18]:
import ROOT
from ROOT import RooFit, RooRealVar, RooDataSet, RooArgList, RooAddPdf, RooGaussian, RooFormulaVar, RooSimultaneous, RooCategory, RooMCStudy

# Create variables
x = RooRealVar("x", "x", -10, 10)
N_total = RooRealVar("N_total", "N_total (N_D+ + N_D-)", 2000, 0, 20000)  # N_total = N_D+ + N_D-
Acp = RooRealVar("Acp", "Acp", 0, -1, 1)  # A_Cp as a fit parameter

# Use Acp and N_total to define the expected signal yields for D+ and D-
Nsig_D_plus = RooFormulaVar("Nsig_D_plus", 
    "0.5 * N_total * (1 + Acp)", 
    RooArgList(N_total, Acp))

Nsig_D_minus = RooFormulaVar("Nsig_D_minus", 
    "0.5 * N_total * (1 - Acp)", 
    RooArgList(N_total, Acp))

# Example Gaussian for D+ and D- signal
mean_D_plus = RooRealVar("mean_D_plus", "Mean D+", 0, -0.5, 0.5)
sigma_D_plus = RooRealVar("sigma_D_plus", "Sigma D+", 1, 0.1, 2)
gauss_D_plus = RooGaussian("gauss_D_plus", "Gaussian PDF for D+", x, mean_D_plus, sigma_D_plus)

mean_D_minus = RooRealVar("mean_D_minus", "Mean D-", 0, -0.5, 0.5)
sigma_D_minus = RooRealVar("sigma_D_minus", "Sigma D-", 1, 0.1, 2)
gauss_D_minus = RooGaussian("gauss_D_minus", "Gaussian PDF for D-", x, mean_D_minus, sigma_D_minus)

# Define extended PDFs for D+ and D-
model_D_plus = RooAddPdf("model_D_plus", "D+ model", RooArgList(gauss_D_plus), RooArgList(Nsig_D_plus))
model_D_minus = RooAddPdf("model_D_minus", "D- model", RooArgList(gauss_D_minus), RooArgList(Nsig_D_minus))

# Create a category to distinguish between D+ and D-
cat = RooCategory("sample", "sample")
cat.defineType("D_plus")
cat.defineType("D_minus")

# Create a simultaneous PDF using the category
sim_model = RooSimultaneous("sim_model", "Simultaneous model", cat)
sim_model.addPdf(model_D_plus, "D_plus")
sim_model.addPdf(model_D_minus, "D_minus")


# Create datasets for D+ and D- (without the Import statement)
data_D_plus = RooDataSet("data_D_plus", "D+ data", RooArgList(x))
data_D_minus = RooDataSet("data_D_minus", "D- data", RooArgList(x))

# Generate example data for D+ and D-
for i in range(520):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_plus.add(RooArgList(x))

for i in range(480):
    x.setVal(ROOT.gRandom.Gaus(0, 1))
    data_D_minus.add(RooArgList(x))

# Combine datasets into one, using RooFit.Import properly
data_combined = RooDataSet("data_combined", "Combined data", RooArgList(x), RooFit.Index(cat),
                           RooFit.Import("D_plus", data_D_plus),
                           RooFit.Import("D_minus", data_D_minus))

# Fit the model to the combined data
fit_result = sim_model.fitTo(data_combined, RooFit.Save())

# RooMCStudy setup: study the Acp and N_total parameters over toy experiments
# We ensure that the category 'sample' is included in the study.
mcstudy = RooMCStudy(sim_model, RooArgList(x), 
                     RooFit.Silence(),  # Suppress fit output
                     RooFit.FitOptions(RooFit.Save()),  # Save fit results
                     RooFit.Binned(False),  # Unbinned fit
                     RooFit.Extended(True),  # Extended likelihood
                     RooFit.Index(cat))  # Index for simultaneous model

# Perform 1000 toy experiments
n_experiments = 1000
mcstudy.generateAndFit(n_experiments)

# Study the Acp and N_total distributions over the toy experiments
# Acp analysis
frame_Acp = mcstudy.plotParam(Acp, RooFit.Bins(50))
frame_Acp.SetTitle("Acp distribution from RooMCStudy")
frame_Acp.Draw()

# N_total analysis
frame_N_total = mcstudy.plotParam(N_total, RooFit.Bins(50))
frame_N_total.SetTitle("N_total distribution from RooMCStudy")
frame_N_total.Draw()

# Fit quality analysis: plot the distribution of fit chi2
frame_chi2 = mcstudy.plotPull(N_total, RooFit.Bins(50))
frame_chi2.SetTitle("Fit quality (pull on N_total)")
frame_chi2.Draw()

# Plot the fit status and errors for further quality checks
fit_status_hist = mcstudy.fitResultPlot(RooFit.FitResultCategory(0))
fit_status_hist.SetTitle("Fit Status (0 means successful fits)")
fit_status_hist.Draw()

fit_errors_hist = mcstudy.plotError(N_total, RooFit.Bins(50))
fit_errors_hist.SetTitle("Error distribution for N_total")
fit_errors_hist.Draw()


runtime_error: bool RooMCStudy::generateAndFit(int nSamples, int nEvtPerSample = 0, bool keepGenData = kFALSE, const char* asciiFilePat = 0) =>
    runtime_error: RooAbsTestStatistic::initSimMode() ERROR, index category of simultaneous pdf is missing in dataset, aborting

 **********
 **   46 **SET PRINT           1
 **********
 **********
 **   47 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 Acp          0.00000e+00  2.00000e-01   -1.00000e+00  1.00000e+00
     2 N_total      2.00000e+03  1.00000e+03    0.00000e+00  2.00000e+04
     3 mean_D_minus   0.00000e+00  1.00000e-01   -5.00000e-01  5.00000e-01
     4 mean_D_plus   0.00000e+00  1.00000e-01   -5.00000e-01  5.00000e-01
     5 sigma_D_minus   1.00000e+00  1.90000e-01    1.00000e-01  2.00000e+00
     6 sigma_D_plus   1.00000e+00  1.90000e-01    1.00000e-01  2.00000e+00
 **********
 **   48 **SET ERR         0.5
 **********
 **********
 **   49 **SET PRINT           1
 **********
 **********
 **   50 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   51 **MIGRAD        3000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
 START MIG